In [1]:
import sys
sys.path.append('/usr/local/lib/python3/dist-packages')

In [2]:
from pywraps2 import S2Point, S2Cell, S2LatLng, S2LatLngRect, S1Angle, S2Loop, S2RegionCoverer, S2Polygon, S2CellId

In [3]:
geojson = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              76.201171875,
              29.869228848968312
            ],
            [
              75.69580078125,
              28.468691297348148
            ],
            [
              77.574462890625,
              28.323724553546015
            ],
            [
              77.7392578125,
              29.35345166863502
            ],
            [
              76.201171875,
              29.869228848968312
            ]
          ]
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              71.575927734375,
              27.430289738862594
            ],
            [
              72.9052734375,
              28.159189634046708
            ],
            [
              73.17993164062499,
              27.381523191705053
            ],
            [
              71.630859375,
              27.401032392938866
            ],
            [
              72.410888671875,
              26.500072915744372
            ],
            [
              74.058837890625,
              27.36201054924028
            ],
            [
              73.5205078125,
              28.98892237190413
            ],
            [
              71.575927734375,
              27.430289738862594
            ]
          ]
        ]
      }
    }
  ]
}

In [4]:
geojson2 = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {
        "stroke": "#555555",
        "stroke-width": 2,
        "stroke-opacity": 1,
        "fill": "#555555",
        "fill-opacity": 0.5
      },
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              77.0496940612793,
              28.45963668793731
            ],
            [
              77.0635986328125,
              28.45043037318006
            ],
            [
              77.05879211425781,
              28.44469488663175
            ],
            [
              77.06789016723633,
              28.436392973656965
            ],
            [
              77.07647323608398,
              28.446506126513928
            ],
            [
              77.08677291870117,
              28.449826652402077
            ],
            [
              77.09672927856445,
              28.450279443308684
            ],
            [
              77.09587097167969,
              28.462956801462703
            ],
            [
              77.07355499267578,
              28.45873118433063
            ],
            [
              77.05767631530762,
              28.467861323783215
            ],
            [
              77.0496940612793,
              28.45963668793731
            ]
          ]
        ]
      }
    }
  ]
}

In [8]:
class GeoJsonParser:
    
    def __init__(self, geojson: dict):
        self.geojson = geojson
        self.polygons = []
        self.loops = []
        
        ## parse the GeoJson
        self.parse_geojson()
        self.get_loops()
        
    def parse_geojson(self):
        features = self.geojson['features']
        for feature in features:
            latlngs = []
            coordinates = feature['geometry']['coordinates']
            for coordinate in coordinates[0][:-1]:
                latlngs.append(S2LatLng.FromDegrees(coordinate[1], coordinate[0]))
            self.polygons.append(latlngs)
            
    def get_loops(self):
        for poly in self.polygons:
            s2loop = []
            for latlng in poly:
                s2loop.append(latlng.ToPoint())
            self.loops.append(s2loop)

In [9]:
parser = GeoJsonParser(geojson2)

In [10]:
parser.loops

[[<pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x4035c5f660> >,
  <pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x4035c5f690> >,
  <pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x4035c5f6c0> >,
  <pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x4035c5f6f0> >,
  <pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x4035c5f720> >,
  <pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x4035c5f750> >,
  <pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x4035c5f780> >,
  <pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x4035c5f7b0> >,
  <pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x4035c5f7e0> >,
  <pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x4035c5f810> >]]

In [12]:
loop1 = S2Loop(parser.loops[0])
# loop2 = S2Loop(parser.loops[1])

In [13]:
poly = S2Polygon(loop1)

In [34]:
min_level = 13
max_level = 13
# max_cells = 10_000

In [35]:
%%time

coverer = S2RegionCoverer()
coverer.set_min_level(min_level)
coverer.set_max_level(max_level)
# coverer.set_max_cells(max_cells)
cells = coverer.GetCovering(poly)

CPU times: user 23.2 ms, sys: 0 ns, total: 23.2 ms
Wall time: 22.5 ms


In [36]:
tokens = []
for cell in cells:
    tokens.append(cell.ToToken())

In [37]:
import pandas as pd

df = pd.DataFrame(tokens, columns=['token'])

In [38]:
df.to_csv('s2_token-13.csv', encoding='utf-8-sig', index=False)